In [ ]:
import wfdb
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import ast
import os
import json

In [ ]:
# imports for filtering

from scipy.signal import butter, lfilter, iirnotch, filtfilt
from scipy.signal import find_peaks

### Code for reading a single signal:

In [ ]:
# Specify the path to your WFDB signal file (e.g., '01000_lr.dat')
signal_file = '01000_lr'

# Use wfdb.rdrecord() to read the signal file
record = wfdb.rdrecord(signal_file)

# Access the signal data and metadata
signal_data = record.p_signal  # NumPy array containing signal data
signal_metadata = record.__dict__  # Dictionary containing metadata

# You can access various metadata properties like sampling frequency, units, etc.
fs = record.fs  # Sampling frequency

# Visualize the signal
plt.figure(figsize=(12, 4))
plt.plot(signal_data)
plt.title('ECG')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

### Filtering of a single signal
#### - Şimal'in eklediği kısım

In [ ]:
# Example: Filtering the ECG signal (Lead I)
ecg_lead = signal_data[:, 0]  # Assuming Lead I is the first column

# High-pass filter to remove baseline wander
b, a = butter(1, 0.5/(0.5*fs), btype='highpass')
filtered_ecg = filtfilt(b, a, ecg_lead)

# R-peak detection
r_peaks, _ = find_peaks(filtered_ecg, height=np.max(filtered_ecg)*0.5, distance=fs*0.6)
rr_intervals = np.diff(r_peaks) / fs  # dividing by sampling frequency to get time in seconds
print("RR Intervals:")
print(rr_intervals)

heart_rate = 60 / rr_intervals  # BPM
print("Heart Rates:")
print(heart_rate)

average_heart_rate = np.mean(heart_rate)

print(f"Average Heart Rate: {average_heart_rate} BPM")
# Visualize filtered signal with R-peaks
plt.figure(figsize=(15, 5))
plt.plot(filtered_ecg, label='Filtered ECG')
plt.plot(r_peaks, filtered_ecg[r_peaks], 'rx', label='R-peaks')
plt.title('Filtered ECG with R-Peak Detection')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

In [ ]:
# Sample ECG data - replace with your actual data

# Sample indices for P waves, QRS complexes, and T waves - replace with actual indices
p_waves = r_peaks - 50  # Example P wave indices
qrs_start = r_peaks - 20  # Example start of QRS complex
qrs_end = r_peaks + 20   # Example end of QRS complex
t_waves = r_peaks + 50  # Example T wave indices

# Plotting
plt.figure(figsize=(15, 5))
plt.plot(filtered_ecg, label='ECG Signal')

# Mark R peaks
plt.plot(r_peaks, filtered_ecg[r_peaks], 'ro', label='R Peaks')

# Mark P waves
plt.plot(p_waves, filtered_ecg[p_waves], 'go', label='P Waves')

# Mark QRS complexes
for start, end in zip(qrs_start, qrs_end):
    plt.axvspan(start, end, color='y', alpha=0.3, label='QRS Complex' if start == qrs_start[0] else "")

# Mark T waves
plt.plot(t_waves, filtered_ecg[t_waves], 'bo', label='T Waves')

plt.title('ECG Signal with R Peaks, P Waves, QRS Complexes, and T Waves')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.show()